In [2]:
from skimage import io
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms

In [3]:
# Read csv
csv_nrg = 'NRG.csv'
csv_rg = 'RG.csv'

df = pd.read_csv('RG.csv')
# df = pd.read_csv('NRG.csv')
ids = df['Eye ID'].tolist()
labels = df['Final Label'].tolist()
ids = [_.split('/')[-1] for _ in ids]
print(ids[:5])
print(labels[:5])

['TRAIN000034.JPG', 'TRAIN000060.JPG', 'TRAIN000068.JPG', 'TRAIN000100.JPG', 'TRAIN000106.JPG']
[1, 1, 1, 1, 1]


In [3]:
class GlauDataset(Dataset):
    def __init__(self, is_train, transform=None):
        self.is_train = is_train
        self.transform = transform

        df = pd.read_csv('RG.csv')
        ids = df['Eye ID'].tolist()
        labs = df['Final Label'].tolist()
        ids = [_.split('/')[-1] for _ in ids]

        df_nrg = pd.read_csv('NRG.csv')
        ids_nrg = df_nrg['Eye ID'].tolist()
        labs_nrg = df_nrg['Final Label'].tolist()
        ids_nrg = [_.split('/')[-1] for _ in ids_nrg]

        num_rg_train = int(len(ids) * 0.8)
        num_nrg_train = int(len(ids_nrg) * 0.8)

        self.train_ids = ids[:num_rg_train] + ids_nrg[:num_nrg_train]
        self.train_labs = labs[:num_rg_train] + labs_nrg[:num_nrg_train]

        self.val_ids = ids[num_rg_train:] + ids_nrg[num_nrg_train:]
        self.val_labs = labs[num_rg_train:] + labs_nrg[num_nrg_train:]


    def __len__(self):
        if self.is_train:
            return len(self.train_ids)
        else:
            return len(self.val_ids)

    def __getitem__(self, idx):
        if self.is_train:
            img = io.imread(self.train_ids[idx])
            lab = self.train_labs[idx]
            img = self.transform(img)
        else:
            img = io.imread(self.val_ids[idx])
            lab = self.val_labs[idx]
            img = torch.from_numpy(img[None])
        
        lab = torch.from_numpy(np.array([lab], dtype=np.float32))
        return  img, lab

In [4]:
class ResNet50(nn.Module):
    def __init__(self):
        super().__init__()
        # model = torchvision.models.resnet50(pretrained=True)
        self.model = torchvision.models.resnet50(weights=None)
        # model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")     
        self.model.fc = nn.Linear(2048, 1)

    def forward(self, x):
        return self.model(x).squeeze()

In [6]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set hyperparameters
num_epochs = 10
batch_size = 2
learning_rate = 0.001

# Initialize transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    # transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the ImageNet Object Localization Challenge dataset
# train_dataset = torchvision.datasets.ImageFolder(
#     root='/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train', 
#     transform=transform
# )
train_dataset = GlauDataset(is_train=True, transform=transform)
val_dataset = GlauDataset(is_train=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=10)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=10)

# Load the ResNet50 model
# model = torchvision.models.resnet50(pretrained=True)
# model = torchvision.models.resnet50(weights=None)
# model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")
model = ResNet50()

# Parallelize training across multiple GPUs
# model = torch.nn.DataParallel(model)

# Set the model to run on the device
model = model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model...
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Move input and label tensors to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

print(f'Finished Training, Loss: {loss.item():.4f}')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'GlauDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


KeyboardInterrupt: 